In [1]:
# Crop Prediction using Machine Learning and Firebase
# This notebook implements a machine learning model for crop prediction using a decision tree classifier. It preprocesses data, trains the model, makes predictions, and sends the results to Firebase.

## Steps:
# 1. Import Libraries
# 2. Define Functions for Preprocessing, Training, and Prediction
# 3. Integrate with Firebase
# 4. Run the Main Script


In [2]:
# Importing required libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
from firebase import firebase


In [3]:
## Data Preprocessing
# This function loads the dataset, encodes labels, splits the data into training and testing sets, and scales the features for the model.


In [4]:
def preprocess_data(filepath):
    data = pd.read_csv(filepath)
    print("First row of data:")
    print(data.head(1))

    # Encode target variable
    le = LabelEncoder()
    data['label_encoded'] = le.fit_transform(data['label'])
    y = data['label_encoded']
    X = data.drop(columns=['label', 'label_encoded'])

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Scale features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test, le, scaler


In [5]:
## Model Training and Evaluation
# This function trains a decision tree classifier, evaluates it using test data, and prints the classification report and accuracy score.


In [6]:
def train_and_evaluate_model(X_train, X_test, y_train, y_test, le):
    clf = DecisionTreeClassifier()
    clf.fit(X_train, y_train) #for training model 

    # Predict and evaluate
    y_pred = clf.predict(X_test)  #prediction
    print("Classification Report:")
    print(classification_report(y_test, y_pred, target_names=le.classes_))
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model Accuracy: {accuracy * 100:.2f}%")

    return clf


In [7]:
## User Input Validation
# This function collects valid user inputs for prediction. It ensures that inputs are within specified ranges.


In [8]:
def get_valid_input(prompt, min_val, max_val):
    while True:
        try:
            value = float(input(prompt))
            if min_val <= value <= max_val:
                return value
            else:
                print(f"Please enter a value between {min_val} and {max_val}.")
        except ValueError:
            print("Invalid input. Please enter a numeric value.")


In [9]:
## Prediction and Firebase Integration
# This function:
# 1. Collects user inputs for prediction.
# 2. Uses the trained model to make predictions.
# 3. Sends the prediction result to Firebase.


In [10]:
def predict_and_send_to_firebase(clf, scaler, le, firebase_url):
    # Define ranges
    input_ranges = {
        "Air Humidity (%)": (10, 90),
        "Air Temperature (°C)": (10, 45),
        "Soil pH": (4, 9),
        "Rainfall (mm)": (0, 300),
    }

    # Gather user inputs
    inputs = [get_valid_input(f"Enter {key}: ", *range_) for key, range_ in input_ranges.items()]
    predict_input_scaled = scaler.transform([inputs])

    # Make prediction
    predicted_label_encoded = clf.predict(predict_input_scaled)
    predicted_crop = le.inverse_transform(predicted_label_encoded)[0]
    print(f"The predicted crop is: {predicted_crop}")

    # Connect to Firebase and send prediction
    try:
        firebase_connection = firebase.FirebaseApplication(firebase_url)
        firebase_connection.put('/croppredicted', 'crop', predicted_crop)
        print("Prediction successfully sent to Firebase!")
    except Exception as e:
        print(f"Error sending data to Firebase: {e}")


In [11]:
## Main Script
# This section combines all the defined functions to:
# 1. Preprocess data.
# 2. Train and evaluate the model.
# 3. Interact with Firebase for real-time data and predictions.


In [12]:
def main():
    # Filepath and Firebase URL
    filepath = 'cpdata1.csv'  # Replace with your actual file path
    firebase_url = 'https://cropit-eb156.firebaseio.com/'

    # Preprocess data
    X_train, X_test, y_train, y_test, le, scaler = preprocess_data(filepath)

    # Train and evaluate the model
    clf = train_and_evaluate_model(X_train, X_test, y_train, y_test, le)

    # Retrieve Firebase data
    firebase_connection = firebase.FirebaseApplication(firebase_url)
    tp = firebase_connection.get('/Realtime', None)

    if tp is not None:
        predict_and_send_to_firebase(clf, scaler, le, firebase_url)
    else:
        print("Failed to retrieve data from Firebase.")


In [ ]:
# Execute the main function
if __name__ == "__main__":
    main()


First row of data:
   temperature   humidity        ph    rainfall label
0    20.879744  82.002744  6.502985  202.935536  rice
Classification Report:
              precision    recall  f1-score   support

Adzuki Beans       1.00      1.00      1.00        29
  Black gram       1.00      0.96      0.98        26
    Chickpea       1.00      1.00      1.00        27
     Coconut       1.00      1.00      1.00        30
      Coffee       0.82      0.90      0.86        31
      Cotton       0.80      0.83      0.82        24
  Ground Nut       0.86      0.91      0.89        34
        Jute       0.91      0.71      0.80        28
Kidney Beans       1.00      1.00      1.00        22
      Lentil       0.93      0.96      0.95        28
  Moth Beans       0.91      0.88      0.89        24
   Mung Bean       1.00      1.00      1.00        40
        Peas       1.00      1.00      1.00        27
 Pigeon Peas       0.83      0.81      0.82        31
      Rubber       0.97      0.94      